In [31]:
from fastbook import *
from fastai.tabular.all import *
import os
import nfl_data_py as nfl
import numpy as np
import pandas as pd

In [5]:
# Calculate Expected Pass Rate

In [11]:
# Set CONSTANTS
SEASONS = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]

In [22]:
# get play by play for season
cols = ['game_id',
        'play_id',
        'epa',
        'season_type',
        'season','week',
        'drive','series',
        'down',
        'ydstogo',
        'yards_gained',
        'wp',
        'ep',
        'yardline_100',
        'home_team',
        'away_team',
        'posteam','posteam_score',
        'defteam','defteam_score',
        'game_seconds_remaining',
        'game_half',
        'fixed_drive_result',
        'play_type',
        'penalty_team',
        'penalty_type',
        'penalty_yards']

df_pbp = nfl.import_pbp_data(years=SEASONS, columns=cols, downcast=True, cache=False)

2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
Downcasting floats.


In [55]:
df_pbp['score_diff'] = df_pbp.posteam_score - df_pbp.defteam_score

In [56]:
is_down = df_pbp['down'].isin([1,2,3,4])
is_runpass = df_pbp['play_type'].isin(['run','pass'])


df = df_pbp.loc[(is_runpass & is_down), ['play_type','season','season_type','score_diff','posteam_score','defteam_score','down','ydstogo','yardline_100','game_half','game_seconds_remaining']].copy()
df

,play_type,season,season_type,score_diff,posteam_score,defteam_score,down,ydstogo,yardline_100,game_half,game_seconds_remaining
2,run,2007,REG,0.0,0.0,0.0,1.0,10.0,64.0,Half1,3588.0
3,pass,2007,REG,0.0,0.0,0.0,2.0,9.0,63.0,Half1,3548.0
5,run,2007,REG,0.0,0.0,0.0,1.0,15.0,54.0,Half1,3498.0
6,pass,2007,REG,0.0,0.0,0.0,2.0,17.0,56.0,Half1,3457.0
7,pass,2007,REG,0.0,0.0,0.0,3.0,17.0,56.0,Half1,3450.0
...,...,...,...,...,...,...,...,...,...,...,...
723338,pass,2021,POST,-3.0,20.0,23.0,1.0,10.0,75.0,Half2,85.0
723339,pass,2021,POST,-3.0,20.0,23.0,1.0,10.0,58.0,Half2,79.0
723340,pass,2021,POST,-3.0,20.0,23.0,2.0,1.0,49.0,Half2,54.0
723341,run,2021,POST,-3.0,20.0,23.0,3.0,1.0,49.0,Half2,48.0


In [70]:
df.play_type.value_counts()

pass    294687
run     209957
Name: play_type, dtype: int64

In [71]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))
# splits = TrainTestSplitter(test_size=0.2, random_state=None, stratify='play_type',
#                     train_size=None, shuffle=True)

In [77]:
to = TabularPandas(df, procs=[Categorify, FillMissing, Normalize],
                   cat_names = ['season', 'season_type','down','game_half'],
                   cont_names = ['score_diff','posteam_score','defteam_score','ydstogo','yardline_100','game_seconds_remaining'],
                   y_names='play_type',
                   splits=splits)

In [78]:
dls = to.dataloaders(bs=64)

In [79]:
dls.show_batch()

,season,season_type,down,game_half,score_diff,posteam_score,defteam_score,ydstogo,yardline_100,game_seconds_remaining,play_type
0,2018,REG,1.0,Half1,-7.000000e+00,-2.455636e-08,7.000000e+00,10.0,75.000000,3170.000008,run
1,2013,REG,2.0,Half2,8.000000e+00,1.900000e+01,1.100000e+01,3.0,70.000000,860.999977,run
2,2012,REG,1.0,Half2,-3.000000e+00,1.000000e+01,1.300000e+01,10.0,64.000000,487.999964,run
3,2016,REG,1.0,Half2,-1.300000e+01,-2.455636e-08,1.300000e+01,1.0,1.000002,892.999994,run
4,2008,REG,3.0,Half2,7.000000e+00,1.000000e+01,3.000000e+00,22.0,92.000001,478.000025,run
5,2012,REG,1.0,Half2,2.300000e+01,3.600000e+01,1.300000e+01,2.0,1.999999,534.000009,run
6,2017,REG,4.0,Half2,3.500000e+01,4.200000e+01,7.000000e+00,5.0,32.000000,345.000049,pass
7,2015,REG,1.0,Half2,-1.000000e+01,1.300000e+01,2.300000e+01,10.0,10.999999,185.000025,pass
8,2020,REG,1.0,Half2,7.000000e+00,2.400000e+01,1.700000e+01,10.0,89.000000,736.999996,pass
9,2016,REG,2.0,Half1,3.686401e-08,-2.455636e-08,1.582168e-08,5.0,65.999999,3552.999959,pass


In [80]:
learn = tabular_learner(dls, metrics=accuracy)

In [81]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,0.591474,0.581468,0.678236,00:57
1,0.576340,0.570082,0.688719,00:57
2,0.572834,0.566970,0.692028,00:58
3,0.573187,0.565243,0.693068,00:57
4,0.570778,0.563083,0.694376,00:57
5,0.572023,0.560917,0.693752,00:59
6,0.579628,0.560453,0.695436,00:56
7,0.567743,0.559554,0.696596,00:55
8,0.563036,0.558756,0.697081,00:55
9,0.561445,0.560980,0.694228,00:55


In [82]:
learn.show_results()

,season,season_type,down,game_half,score_diff,posteam_score,defteam_score,ydstogo,yardline_100,game_seconds_remaining,play_type,play_type_pred
0,10.0,2.0,3.0,1.0,-0.502843,-1.074228,-0.458176,-0.658637,1.307757,1.038418,0.0,0.0
1,13.0,2.0,3.0,1.0,-1.145415,-1.074228,0.225941,-0.913625,-0.161187,0.807923,0.0,0.0
2,3.0,2.0,3.0,2.0,-1.420802,-1.074228,0.519133,0.361315,-1.018072,-0.511923,0.0,0.0
3,3.0,2.0,1.0,1.0,0.506912,-0.334545,-0.849099,0.361315,0.981325,1.206746,0.0,1.0
4,7.0,2.0,1.0,2.0,-0.778231,-0.017538,0.812326,0.361315,0.981325,-0.415326,0.0,0.0
5,12.0,2.0,1.0,2.0,1.149483,0.722145,-0.555907,0.361315,0.287657,-0.137010,1.0,1.0
6,6.0,2.0,2.0,1.0,-0.135660,-0.017538,0.128210,0.361315,0.736501,0.147043,0.0,0.0
7,12.0,2.0,2.0,2.0,0.874095,1.250490,0.225941,-0.148661,-0.732444,-1.494156,1.0,1.0
8,3.0,2.0,3.0,1.0,0.139728,-1.074228,-1.142292,-1.423601,0.450873,0.943733,0.0,0.0
